In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
#from tabulate import tabulate

In [ ]:
#!pip install pymysql

In [ ]:
#!Pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from PIL import Image
import datetime as DT
import time

In [ ]:
from re import search
import csv
import os
import requests
from sqlalchemy import create_engine

import pymysql

In [ ]:
# Credentials to database connection
hostname="mysql_db_ip_address"
dbname="db_name"
uname="db_user_name"
pwd="db_password"
ethTxTable="ethtrans2"
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [ ]:
chrome_driver_path='path\to\chromedriver.exe'
eth_wallet='eth wallet'

In [ ]:
csv_temp_file='temp file.csv'
csv_final_file='final file.csv'

In [ ]:
def getPayoutFromEthermine(wallet_):
    url_= 'https://ethermine.org/miners/'+ wallet_+'/payouts'
    #s=Service('D:\\libs\\chromedriver.exe')
    #driver = webdriver.Chrome(service=s)
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    driver.get(url_)
    time.sleep(10)

    content=BeautifulSoup(driver.page_source, 'lxml')
    table = content.find_all('table')[0]
    rows = []

    for tr in table.find_all('tr'):
        rows.append([td.text for td in tr.find_all(['th', 'td'])])
        
    cols=['PaidOn','FromBlock','ToBlock','Duration','Amount','TxHash']
    df = pd.DataFrame(rows[1:], columns=cols)
    df.PaidOn =df.PaidOn.str.replace('.','',regex=True)
    df['PaidOnDate'] = pd.to_datetime(df.PaidOn, format='%d %b %Y %H:%M', errors='raise').copy()
    return df

In [ ]:
def insertToDB(df_,insert_mode='append'):
    # Convert dataframe to sql table                                   
    df_.to_sql(ethTxTable, engine,if_exists=insert_mode, index=False)

In [ ]:
def getLastPayoutDB():
    df2 = pd.read_sql('SELECT * FROM ' + ethTxTable + ' where paidOnDate = (select max(paidOnDate) from ' + ethTxTable + ')', con=engine)
    return df2.PaidOn[0]

In [ ]:

def writeCsvFile(fname,data_):           
    outFile=fname        
    with open(outFile, 'w',newline='' ) as csvfile:
        csvwriter = csv.writer(csvfile)        
        for line in data_:            
            #print(line)
            csvwriter.writerow(line)


In [ ]:
def getNewRow(df_):
    lastPayout = getLastPayoutDB()
    print(lastPayout)
    r_index = 0
    for index, row in df_.iterrows():
        if (row.PaidOn == lastPayout):
            print(index)
            r_index = index
            break
        else :
            r_index = index
            print (row.PaidOn+ ' '+ lastPayout)
    print('new Row' + str(r_index))    
    return df_[:r_index]      

In [ ]:
def getTransactionToDb(first_time=False):
    dfx =  getPayoutFromEthermine(eth_wallet)
    if(first_time == True):
        insertToDB(dfx)
    else:
        try:            
            newRow=getNewRow(dfx)            
            insertToDB(newRow)
        except Exception as e:
            print('No new Transaction')
            print (e)

In [ ]:
def getDataFromDb():
    df2 = pd.read_sql('SELECT * FROM ' + ethTxTable , con=engine)
    df2.Amount = df2.Amount.astype('float64')
    df2['Month'] = df2['PaidOnDate'].dt.strftime("%Y-%m")
    df2['Year'] = df2['PaidOnDate'].dt.strftime("%Y")
    #data_g =df2.groupby('Month').agg({'Amount': [ 'max','sum','count']})
    #data_gYear =df2.groupby('Year').agg({'Amount': [ 'max','sum','count']})
    p = df2.pivot_table(['Amount'], index='Month', aggfunc=['count','sum'],margins=True)
    p.columns=['Transfer_time','Amt_Eth']
    p = p.sort_values(by='Month', ascending = False)
    p['Amt2'] = p['Amt_Eth'].astype(str)
    p.reset_index(inplace = False)
    p.T.drop('Amt2').to_csv(csv_temp_file,index=True,float_format='%.5f') 

In [ ]:
def createCsvFile():
    with open(csv_temp_file) as file:
        content = file.readlines()
        header = content[:1]
        rows = content[1:]
        rows=content
        #print(header)
        #print(rows)
        new_content=[] 
        #new_content.append(header)

        for i in range(len(rows)):
            #print (str(i) + rows[i])
            r =rows[i]
            cls = r.split(',')
            if i == 2:
                res = [str("{:.5f}".format(float(cls[value]))) for value in range(1,len(cls))]
                res.insert(0,cls[0])
                new_content.append(res)
            else:
                new_content.append(cls)

        writeCsvFile(csv_final_file,new_content) 

In [ ]:
getTransactionToDb(False)
getDataFromDb()
createCsvFile()

In [ ]:
#!jupyter nbconvert --to script EtherminePayoutMysql.ipynb